In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import npc_lims
from npc_sessions import DynamicRoutingSession
from dynamic_routing_analysis import decoding_utils

import os

%load_ext autoreload
%autoreload 2

In [ ]:
# DR_ephys_sessions = tuple(s for s in npc_lims.get_session_info(is_ephys=True) 
#                           if s.is_uploaded and s.is_annotated and 
#                           s.project=='DynamicRouting')

#load Templeton sessions
ephys_sessions = tuple(s for s in npc_lims.get_session_info(is_ephys=True, is_uploaded=True, is_annotated=True)) #, project='DynamicRouting', project='TempletonPilotSession'))


In [ ]:
ephys_sessions[3]

In [ ]:
# #find all DR sessions with ephys
# DR_ephys_sessions = tuple(s for s in npc_lims.get_session_info(is_ephys=True) 
                        #   if s.is_uploaded and s.is_annotated and 
                        #   s.project=='DynamicRouting')

savepath=r'\\allen\programs\mindscope\workgroups\templeton\TTOC\decoding results\forecasting_3'
filename='forecasting_3.pkl'

except_list={}

spikes_binsize=0.1
spikes_time_before=0.5
spikes_time_after=0.5

decoder_binsize=0.2
decoder_time_before=0.2
decoder_time_after=0.1

params={
    'trnum':'all',
    # 'n_units':[5,10,20,50,100,'all'],
    'n_units':['all'],
    'u_min':20,
    'n_repeats':1,
    'spikes_binsize':spikes_binsize,
    'spikes_time_before':spikes_time_before,
    'spikes_time_after':spikes_time_after,
    'decoder_binsize':decoder_binsize,
    'decoder_time_before':decoder_time_before,
    'decoder_time_after':decoder_time_after,
    'balance_labels':0,
    'savepath':savepath,
    'filename':filename,
    'use_structure_probe':True,
    'crossval':'forecast_train_3',
    'all_areas':True,
    'labels_as_index':True,
    'decoder_type':'linearSVC',
    'generate_labels':True,
}


for ephys_session in ephys_sessions[:]:
    try:
        session = DynamicRoutingSession(ephys_session.id)
        print(session.id+' loaded')
        if 'structure' in session.electrodes[:].columns:
            decoding_utils.decode_context_from_units(session,params)
        else:
            print('no structure column found in electrodes table, moving to next recording')
        session=[]
    except Exception as e:
        except_list[session.id]=repr(e)
